In [1]:
!pip install datasets pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-c

In [2]:
from datasets import load_dataset
import pandas as pd
import json

In [3]:
dataset = load_dataset("spider")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [4]:
# Show a few examples to understand the structure
dataset["train"].shuffle(seed=42).select(range(3))

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 3
})

In [7]:
import json

tables_path = "/content/drive/Shareddrives/DATA266_Project/Data/spider_data/tables.json"

with open(tables_path) as f:
    tables_data = json.load(f)

In [8]:
# Parse schema into db_id → readable schema string
def build_schema_index(tables_json):
    db_schemas = {}
    for db in tables_json:
        db_id = db["db_id"]
        schema_lines = []
        for table_name, cols in zip(db["table_names_original"], db["column_names_original"]):
            table_columns = []
            for col_idx, (table_idx, col_name) in enumerate(db["column_names_original"]):
                if table_idx == db["table_names_original"].index(table_name) and col_name != "*":
                    col_type = db["column_types"][col_idx]
                    table_columns.append(f"{col_name}:{col_type}")
            if table_columns:
                schema_lines.append(f"{table_name}({', '.join(table_columns)})")
        db_schemas[db_id] = "\n".join(schema_lines)
    return db_schemas

db_schemas = build_schema_index(tables_data)

In [9]:
def create_instruction_sample(example):
    question = example["question"]
    sql = example["query"]
    db_id = example["db_id"]
    schema = db_schemas.get(db_id, "")

    prompt = f"""### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
{question}

### Schema:
{schema}

### SQL:"""

    return {
        "prompt": prompt,
        "response": sql,
        "db_id": db_id
    }

In [10]:
# Apply the prompt generator to the full training set
train_dataset = dataset["train"].map(create_instruction_sample)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [11]:
# Show a few random formatted examples
for i in range(3):
    print("Prompt:\n", train_dataset[i]["prompt"])
    print("Response:\n", train_dataset[i]["response"])
    print()


Prompt:
 ### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
How many heads of the departments are older than 56 ?

### Schema:
department(Department_ID:number, Name:text, Creation:text, Ranking:number, Budget_in_Billions:number, Num_Employees:number)
head(head_ID:number, name:text, born_state:text, age:number)
management(department_ID:number, head_ID:number, temporary_acting:text)

### SQL:
Response:
 SELECT count(*) FROM head WHERE age  >  56

Prompt:
 ### Instruction:
Generate an SQL query for the following question based on the database schema.

### Question:
List the name, born state and age of the heads of departments ordered by age.

### Schema:
department(Department_ID:number, Name:text, Creation:text, Ranking:number, Budget_in_Billions:number, Num_Employees:number)
head(head_ID:number, name:text, born_state:text, age:number)
management(department_ID:number, head_ID:number, temporary_acting:text)

### SQL:
Response:
 SE

In [13]:
output_path = "/content/drive/Shareddrives/DATA266_Project/Data/spider_data/spider_instruction_data.json"

# Convert to list of dictionaries
formatted_data = [
    {
        "prompt": example["prompt"],
        "response": example["response"],
        "db_id": example["db_id"]
    }
    for example in train_dataset
]

with open(output_path, "w") as f:
    json.dump(formatted_data, f, indent=2)

print(f"Saved instruction-style training data to {output_path}")

Saved instruction-style training data to /content/drive/Shareddrives/DATA266_Project/Data/spider_data/spider_instruction_data.json
